# MDP Genomic Analysis

This notebook demonstrates how to perform a genomic analysis using `libsemx` with the Maize Diversity Project (MDP) datasets. We will estimate the heritability of a trait (Ear Height) using a Genomic Relationship Matrix (GRM) derived from SNP markers.

## Prerequisites

Ensure `libsemx` is installed. If running from source, we add the build directory to the path.

In [1]:
import sys
import os
import importlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Add the build directory to sys.path if running from source
sys.path.append("../../build")
sys.path.append("../../python")

# Force reload of the C++ extension
if '_libsemx' in sys.modules:
    del sys.modules['_libsemx']
import _libsemx
import semx

importlib.reload(semx.model)
importlib.reload(semx)

print(f"libsemx version: {semx.__version__ if hasattr(semx, '__version__') else 'unknown'}")

libsemx version: 0.0.0.dev0


## 1. Load and Preprocess Data

We load the numeric SNP data (`mdp_numeric.csv`) and the phenotype data (`mdp_traits.csv`).
*   `mdp_numeric.csv`: Contains SNP markers (0, 1, 2) for each taxa.
*   `mdp_traits.csv`: Contains phenotypes like Ear Height (`EarHT`).

We need to:
1.  Merge the datasets on the taxa identifier.
2.  Extract the marker matrix ($M$) and the phenotype vector ($y$).
3.  Ensure the order of rows in $M$ matches the order of rows in the dataframe used for fitting.

In [2]:
# Load datasets
numeric_df = pd.read_csv("../../data/mdp_numeric.csv")
traits_df = pd.read_csv("../../data/mdp_traits.csv")

print("Numeric DF shape:", numeric_df.shape)
print("Traits DF shape:", traits_df.shape)

# Rename 'Taxa' to 'taxa' in traits_df for consistency if needed, 
# but numeric_df has 'taxa' and traits_df has 'Taxa'.
traits_df = traits_df.rename(columns={"Taxa": "taxa"})

# Ensure 'taxa' is string in both to avoid merge issues (e.g. "811" vs 811)
numeric_df["taxa"] = numeric_df["taxa"].astype(str)
traits_df["taxa"] = traits_df["taxa"].astype(str)

# Merge datasets
# We use inner join to keep only individuals with both genotype and phenotype
merged_df = pd.merge(numeric_df, traits_df, on="taxa", how="inner")

# Drop rows with missing phenotype (EarHT)
merged_df = merged_df.dropna(subset=["EarHT"])

print("Merged DF shape:", merged_df.shape)

# Sort by taxa to ensure deterministic order
merged_df = merged_df.sort_values("taxa").reset_index(drop=True)

# Add a global grouping variable for GRM (all individuals in one group)
# This is required for libsemx when using a dense covariance matrix (GRM) 
# that spans all individuals.
merged_df["_global"] = 1.0

# Extract Markers
# Identify marker columns: they are in numeric_df but not 'taxa'
marker_cols = [c for c in numeric_df.columns if c != "taxa"]
M = merged_df[marker_cols].values.astype(float)

print(f"Marker Matrix shape: {M.shape}")

# Standardize Phenotype
# This helps with convergence
merged_df["EarHT_std"] = (merged_df["EarHT"] - merged_df["EarHT"].mean()) / merged_df["EarHT"].std()

print(merged_df[["taxa", "EarHT", "EarHT_std"]].head())

Numeric DF shape: (281, 3094)
Traits DF shape: (301, 4)
Merged DF shape: (279, 3097)
Marker Matrix shape: (279, 3093)
    taxa  EarHT  EarHT_std
0  33-16  64.75   0.159543
1  38-11  92.25   1.513962
2   4226  65.50   0.196482
3   4722  81.13   0.966285
4   A188  27.50  -1.675079


## 2. Define Genomic Model

We define a mixed model:
$$ y = \mu + u + e $$
where:
*   $y$ is the standardized Ear Height.
*   $\mu$ is the fixed intercept.
*   $u \sim N(0, \sigma_g^2 K)$ is the random polygenic effect, with $K$ being the Genomic Relationship Matrix (GRM) computed from markers $M$.
*   $e \sim N(0, \sigma_e^2 I)$ is the residual error.

In `libsemx`, we specify this using:
1.  `equations`: `EarHT_std ~ 1` (fixed intercept).
2.  `genomic`: Defines the covariance structure `polygenic` using the marker matrix $M$.
3.  `random_effects`: Links the grouping variable `taxa` to the `polygenic` covariance. Note that since each row is a unique taxa, this effectively models the polygenic effect for each individual.

In [3]:
# Define the model
model = semx.Model(
    equations=["EarHT_std ~ 1"],
    families={"EarHT_std": "gaussian"},
    # Define the genomic covariance structure named 'polygenic'
    # structure='grm' tells libsemx to compute VanRaden GRM from markers
    genomic={
        "polygenic": {
            "markers": M,
            "structure": "grm"
        }
    },
    # Define the random effect
    # We use '_global' as the grouping variable to treat all individuals as a single group.
    # This matches the dimension of the GRM (N x N).
    random_effects=[{
        "name": "u",
        "variables": ["_global"],
        "covariance": "polygenic"
    }]
)

print("Model defined successfully.")

Model defined successfully.


## 3. Fit Model and Analyze Results

We fit the model and inspect the variance components to calculate heritability ($h^2$).

$$ h^2 = \frac{\sigma_g^2}{\sigma_g^2 + \sigma_e^2} $$

In [4]:
# Fit the model
# Note: This might take a moment as it involves dense matrix operations for the GRM
fit = model.fit(merged_df)

# Display Summary
print(fit.summary())

# Extract Variance Components
# The parameter names depend on the internal ID generation.
# Usually, the genomic variance will be associated with the 'polygenic' covariance.
# The residual variance is associated with the outcome family.

params = fit.parameter_estimates
print("\nParameter Estimates:")
for k, v in params.items():
    print(f"{k}: {v:.4f}")

# Calculate Heritability
# We need to identify which parameter is sigma_g^2 and which is sigma_e^2.
# Based on libsemx naming convention:
# - Residual variance for 'EarHT_std' is usually named 'EarHT_std_sigma' or similar (actually it's often just the family parameter).
# - Genomic variance is the parameter for the 'polygenic' covariance.

# Let's inspect the parameter names from the summary or fit object
# For now, we'll try to deduce them.
# The covariance parameter for 'polygenic' (structure='grm') is a single scalar (variance component).
# Let's look for parameters starting with 'polygenic'.

sigma_g2 = next((v for k, v in params.items() if k.startswith("polygenic")), 0.0)
# For Gaussian family, the dispersion parameter is the residual variance (sigma^2) or standard deviation?
# In libsemx, Gaussian family has a 'sigma' parameter which is standard deviation, or 'phi' (dispersion)?
# Let's check the parameter keys.
sigma_e_param = next((v for k, v in params.items() if "EarHT_std" in k and ("sigma" in k or "phi" in k)), 1.0)
# If it's sigma (std dev), we square it.
sigma_e2 = sigma_e_param ** 2

h2 = sigma_g2 / (sigma_g2 + sigma_e2)
print(f"\nEstimated Heritability (h^2): {h2:.4f}")

Optimization converged: True
Iterations: 6
Log-likelihood: -379.706
AIC: 763.4, BIC: 770.7

                                  Estimate  Std.Error       z-value   P(>|z|)
beta_EarHT_std_on__intercept -3.675545e-10   0.059868 -6.139368e-09  1.000000
polygenic_0                   2.332975e-01   0.069308  3.366118e+00  0.000762

Parameter Estimates:
beta_EarHT_std_on__intercept: -0.0000
polygenic_0: 0.2333

Estimated Heritability (h^2): 0.1892


## 4. Using Precomputed Covariance Matrices

In some cases, you may want to compute the Genomic Relationship Matrix (GRM) or another kernel externally (e.g., using specialized software like PLINK or GCTA, or a custom kernel function) and pass it directly to `libsemx`.

This avoids recomputing the GRM from raw markers every time and allows for greater flexibility.

Here, we manually compute the VanRaden GRM using numpy and pass it to the model.

$$ K = \frac{ZZ'}{2 \sum p_i(1-p_i)} $$

where $Z$ is the centered marker matrix.

In [5]:
# 1. Compute GRM manually
# Center the markers
M_mean = np.mean(M, axis=0)
Z = M - M_mean

# Calculate denominator: 2 * sum(p_i * (1 - p_i))
# p_i is the allele frequency of the reference allele.
# Since markers are 0, 1, 2, the frequency p = mean / 2
p = M_mean / 2.0
denom = 2.0 * np.sum(p * (1.0 - p))

# Compute K
K = np.dot(Z, Z.T) / denom

print(f"Computed GRM shape: {K.shape}")
print(f"GRM diagonal mean: {np.mean(np.diag(K)):.4f}")

# 2. Define model with precomputed kernel
# We use the 'data' key instead of 'markers' in the genomic spec.
# 'structure' can still be 'grm' (conceptually) or 'custom', 
# but when 'data' is passed, libsemx treats it as the covariance matrix itself.

model_precomputed = semx.Model(
    equations=["EarHT_std ~ 1"],
    families={"EarHT_std": "gaussian"},
    genomic={
        "polygenic_pre": {
            "data": K,          # Pass the precomputed matrix here
            "structure": "grm"  # Label for the structure type
        }
    },
    random_effects=[{
        "name": "u_pre",
        "variables": ["_global"],
        "covariance": "polygenic_pre"
    }]
)

# 3. Fit the model
fit_pre = model_precomputed.fit(merged_df)

print("\n--- Precomputed Kernel Model Summary ---")
print(fit_pre.summary())

# Compare Heritability
params_pre = fit_pre.parameter_estimates
sigma_g2_pre = next((v for k, v in params_pre.items() if k.startswith("polygenic_pre")), 0.0)
sigma_e_param_pre = next((v for k, v in params_pre.items() if "EarHT_std" in k and ("sigma" in k or "phi" in k)), 1.0)
sigma_e2_pre = sigma_e_param_pre ** 2
h2_pre = sigma_g2_pre / (sigma_g2_pre + sigma_e2_pre)

print(f"\nHeritability (Precomputed): {h2_pre:.4f}")
print(f"Difference from previous: {abs(h2 - h2_pre):.6f}")

Computed GRM shape: (279, 279)
GRM diagonal mean: 1.8562

--- Precomputed Kernel Model Summary ---
Optimization converged: True
Iterations: 6
Log-likelihood: -379.706
AIC: 763.4, BIC: 770.7

                                  Estimate  Std.Error       z-value   P(>|z|)
beta_EarHT_std_on__intercept -2.047167e-09   0.059868 -3.419441e-08  1.000000
polygenic_pre_0               1.256832e-01   0.037338  3.366118e+00  0.000762

Heritability (Precomputed): 0.1117
Difference from previous: 0.077515
